### Anatomy registration to ephys 
#### This code is designed to guide you through registering an anatmap and applying those channel locations to the data We assume that by now, you have registered the brain to the atlas, traced the tracks and put them into the correct folder in histology, with the correct name format. 

#### now, you need to run this code in "iblenv". executing the first cell will allow you to call a desired anatmap and will format the data correctly for the IBL gui to use. This formatting takes substantial time. 


In [15]:
# add a func to loop over SC recordings (selected somehow by Flora)
import json,glob,sys,datetime
import pandas as pd 
import numpy as np
from pathlib import Path
pinkRig_path= glob.glob(r'C:\Users\*\Documents\Github\PinkRigs')
pinkRig_path = Path(pinkRig_path[0])
sys.path.insert(0, (pinkRig_path.__str__()))
from Processing.pykilo.convert_to_ibl_format import add_anat_to_ibl_format
from Admin.csv_queryExp import queryCSV
from Processing.pyhist.assign_clusters_to_atlas import call_for_anatmap_recordings,save_out_cluster_location,save_to_common_anatmap
#session_list = pd.read_csv(prPath / 'Helpers/sc_selected_recordings.csv')
#session_list = queryCSV(subject='FT009',expDate='2021-01-20',expDef='sparseNoise',checkSpikes='1')
probe = 'probe0'
session_list = call_for_anatmap_recordings(subject='AV028',checkSpikes='1',probe=probe,depth_selection='auto') 
for _,r in session_list.iterrows():
    # read corresponding ephys files 
    corresponding_ephys_json= list((Path(r.expFolder)).glob('ONE_preproc\probe*\_av_rawephys.path*.json'))
    for rec_path in corresponding_ephys_json: 
            # open json to get the actual path
            rec = open(rec_path,)
            rec = json.load(rec)
            rec = Path(rec)
            print(rec.parents[2])
            s=add_anat_to_ibl_format(rec.parents[2],recompute=False)


\\zaru.cortexlab.net\Subjects\AV028\2022-10-26\ephys\AV028_2022-10-26_SparseNoise_part7_g1\AV028_2022-10-26_SparseNoise_part7_g1_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV028\2022-10-26\ephys\AV028_2022-10-26_SparseNoise_part4_g0\AV028_2022-10-26_SparseNoise_part4_g0_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV028\2022-10-26\ephys\AV028_2022-10-26_SparseNoise_part7_g0\AV028_2022-10-26_SparseNoise_part7_g0_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV028\2022-10-26\ephys\AV028_2022-10-26_SparseNoise_part3_g0\AV028_2022-10-26_SparseNoise_part3_g0_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV028\2022-10-26\ephys\AV028_2022-10-26_SparseNoise_part6_g0\AV028_2022-10-26_SparseNoise_part6_g0_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV028\2022-10-26\ephys\AV028_2022-10-26_SparseNoise_part2_g0\AV028_2022-10-26_SparseNoise_part2

### Good job. Now you are ready to use the gui. Print session_list with the below cell and register all your recordings to ephys!
#### cheat to gui:
###### conda activate iblenv
###### cd C:\Users\Flora\Documents\Github\int-brain-lab\iblapps\atlaselectrophysiology
###### python ephys_atlas_gui.py -o True

#### tip: if you don't know which ones you have done already, proceed to the next cell and come back.

In [14]:
session_list


,expDate,expNum,expDef,expDuration,rigName,existBlock,existTimeline,existFrontCam,existSideCam,existEyeCam,...,extractEvents,expFolder,ephysPathProbe0,ephysPathProbe1,subject,probe0_raw,shank_range,depth_range,shank,botrow
7,2021-11-04,8,AP_sparseNoise,633,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\8,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,"{'channels': {'localCoordinates': [[0. 0.], [3...","(0.0, 32.0)","(0.0, 2865.0)",0,0
3,2021-11-04,4,AP_sparseNoise,633,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\4,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,{'channels': {'localCoordinates': [[ 0. 2880...,"(0.0, 32.0)","(2880.0, 5745.0)",0,192
6,2021-11-04,7,AP_sparseNoise,634,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\7,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,{'channels': {'localCoordinates': [[ 200. 2160...,"(200.0, 232.0)","(0.0, 2865.0)",1,0
2,2021-11-04,3,AP_sparseNoise,638,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\3,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,{'channels': {'localCoordinates': [[ 200. 5040...,"(200.0, 232.0)","(2880.0, 5745.0)",1,192
5,2021-11-04,6,AP_sparseNoise,689,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\6,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,{'channels': {'localCoordinates': [[400. 720.]...,"(400.0, 432.0)","(0.0, 2865.0)",2,0
1,2021-11-04,2,AP_sparseNoise,693,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\2,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,{'channels': {'localCoordinates': [[ 400. 3600...,"(400.0, 432.0)","(2880.0, 5745.0)",2,192
4,2021-11-04,5,AP_sparseNoise,632,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\5,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,{'channels': {'localCoordinates': [[ 600. 1440...,"(600.0, 632.0)","(0.0, 2865.0)",3,0
0,2021-11-04,1,AP_sparseNoise,638,zelda-stim4,1,1,1,1,1,...,1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04\1,\\zinu.cortexlab.net\Subjects\FT038\2021-11-04...,NaN,FT038,{'channels': {'localCoordinates': [[ 600. 4320...,"(600.0, 632.0)","(2880.0, 5745.0)",3,192


### Done? Was it tedious...? Well, the hard part is over. Now just run the below code, such that we save a rough channel map 

In [10]:
last_date = np.max(session_list.expDate.values)
for _,r in session_list.iterrows():
    one_path = Path(r.expFolder)  / 'ONE_preproc' / probe
    save_out_cluster_location(one_path) # for the recordings in question
    save_to_common_anatmap(one_path,probe,r.shank,r.botrow,last_date)

## now we apply the contstructed anatmaps for the rest of the data 

In [21]:
# get all recordings 
all_rec_list = queryCSV(subject=r.Subject,expDate='postImplant')
all_rec_list = all_rec_list.dropna(subset='extractSpikes')
all_rec_list = all_rec_list[[rec.extractSpikes[int(probe[-1])*2]=='1' for _,rec in all_rec_list.iterrows()]]
# for each recording search for nearest anatmap available 
anatmap_paths = Path(all_rec_list.iloc[0].expFolder).parents[1]
anatmap_paths = anatmap_paths / 'histology/registered_anatmaps'


for _,r in all_rec_list.iterrows(): 
    my_expDate = datetime.datetime.strptime(r.expDate,'%Y-%m-%d')
    anatmap_dates = [datetime.datetime.strptime(d.name,'%Y-%m-%d') for d in list(anatmap_paths.iterdir())]
    timediffs = [abs(d-my_expDate) for d in anatmap_dates]
    selected_anatmap_date = anatmap_dates[np.argmin(timediffs)]
    selected_anatmaps = list((anatmap_paths / selected_anatmap_date.strftime('%Y-%m-%d')).glob('*%s*.json' % probe))
    one_path = Path(r.expFolder)  / 'ONE_preproc' / probe
    save_out_cluster_location(one_path,anatmap_paths=selected_anatmaps)